In [ ]:
!pip install pytorch_lightning==1.7.5
!pip install timm
!pip install torchmetrics==0.11.4
!pip install -U 'jsonargparse[signatures]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.6/706.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 58.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-man

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random

torch.cuda.empty_cache()

# from lib.pos_embed import interpolate_pos_embed

from pytorch_lightning import LightningModule
from pytorch_lightning.cli import LightningCLI
from pytorch_lightning.utilities.types import STEP_OUTPUT, EPOCH_OUTPUT
from pytorch_lightning.callbacks import Callback
# from lightning.pytorch.cli import ArgsType, LightningCLI
from torchmetrics import Accuracy, ConfusionMatrix, AUROC
from torch.optim import Optimizer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import timm
from timm.data import Mixup
from timm.models import create_model
from timm.optim import create_optimizer_v2
from timm.scheduler import create_scheduler
from timm.scheduler.scheduler import Scheduler
from timm.models.vision_transformer import VisionTransformer
from timm.models.layers import PatchEmbed
from timm.data import ImageDataset, create_transform
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.data.transforms import RandomResizedCropAndInterpolation, ToNumpy, ToTensor

from pathlib import Path
from typing import Optional
from pytorch_lightning import Trainer
from pytorch_lightning import LightningDataModule
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS, EVAL_DATALOADERS

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

from pathlib import Path
# from lib.augment import new_data_aug_generator
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS, EVAL_DATALOADERS
from timm.data import ImageDataset, create_transform
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


from PIL import ImageFilter, ImageOps
from torchvision import transforms


In [ ]:
class GaussianBlur(object):
    """
    Apply Gaussian Blur to the PIL image.
    """

    def __init__(self, p=0.1, radius_min=0.1, radius_max=2.):
        self.prob = p
        self.radius_min = radius_min
        self.radius_max = radius_max

    def __call__(self, img):
        do_it = random.random() <= self.prob
        if not do_it:
            return img

        img = img.filter(
            ImageFilter.GaussianBlur(
                radius=random.uniform(self.radius_min, self.radius_max)
            )
        )
        return img


class Solarization(object):
    """
    Apply Solarization to the PIL image.
    """

    def __init__(self, p=0.2):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            return ImageOps.solarize(img)
        else:
            return img


class GrayScale(object):
    """
    Apply Grayscale transformation to the PIL image.
    """

    def __init__(self, p=0.2):
        self.p = p
        self.transf = transforms.Grayscale(3)

    def __call__(self, img):
        if random.random() < self.p:
            return self.transf(img)
        else:
            return img


def new_data_aug_generator(args=None):
    img_size = args.input_size
    remove_random_resized_crop = args.src
    mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    primary_tfl = []
    scale = (0.08, 1.0)
    interpolation = 'bicubic'

    if remove_random_resized_crop:
        primary_tfl = [
            transforms.Resize(img_size, interpolation=3),
            transforms.RandomCrop(img_size, padding=4, padding_mode='reflect'),
            transforms.RandomHorizontalFlip()
        ]
    else:
        primary_tfl = [
            RandomResizedCropAndInterpolation(img_size, scale=scale, interpolation=interpolation),
            transforms.RandomHorizontalFlip()
        ]

    secondary_tfl = [transforms.RandomChoice([GrayScale(p=0.8),
                                              Solarization(p=0.8),
                                              GaussianBlur(p=0.8)])]

    if args.color_jitter is not None and not args.color_jitter == 0:
        secondary_tfl.append(transforms.ColorJitter(args.color_jitter, args.color_jitter, args.color_jitter))

    final_tfl = [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=torch.tensor(mean),
            std=torch.tensor(std))
    ]

    return transforms.Compose(primary_tfl + secondary_tfl + final_tfl)

In [ ]:
def interpolate_pos_embed(model, checkpoint_model):
    if 'pos_embed' in checkpoint_model:
        pos_embed_checkpoint = checkpoint_model['pos_embed']
        embedding_size = pos_embed_checkpoint.shape[-1]
        num_patches = model.patch_embed.num_patches
        num_extra_tokens = model.pos_embed.shape[-2] - num_patches
        # height (== width) for the checkpoint position embedding
        orig_size = int((pos_embed_checkpoint.shape[-2] - num_extra_tokens) ** 0.5)
        # height (== width) for the new position embedding
        new_size = int(num_patches ** 0.5)
        # class_token and dist_token are kept unchanged
        if orig_size != new_size:
            print("Position interpolate from %dx%d to %dx%d" % (orig_size, orig_size, new_size, new_size))
            extra_tokens = pos_embed_checkpoint[:, :num_extra_tokens]
            # only the position tokens are interpolated
            pos_tokens = pos_embed_checkpoint[:, num_extra_tokens:]
            pos_tokens = pos_tokens.reshape(-1, orig_size, orig_size, embedding_size).permute(0, 3, 1, 2)
            pos_tokens = torch.nn.functional.interpolate(
                pos_tokens, size=(new_size, new_size), mode='bicubic', align_corners=False)
            pos_tokens = pos_tokens.permute(0, 2, 3, 1).flatten(1, 2)
            new_pos_embed = torch.cat((extra_tokens, pos_tokens), dim=1)
            checkpoint_model['pos_embed'] = new_pos_embed

In [ ]:
class AutismDatasetModule(LightningDataModule):
    def __init__(self,
                 batch_size: int = 16,
                 num_workers: int = 4,
                 data_root: str = "/kaggle/input/autism-gurv/data",
                 input_size: int = 224,
                 color_jitter: float = 0.3,
                 three_augment: bool = True,
                 src: bool = False  # simple random crop
                 ):
        super(AutismDatasetModule, self).__init__()
        self.save_hyperparameters()
        self.data_path = Path(self.hparams.data_root)
        self.train_transforms = self.build_transform(is_train=True)
        self.eval_transforms = self.build_transform(is_train=False)
        if self.hparams.three_augment:
            self.train_transforms = new_data_aug_generator(self.hparams)
        self.class_map = {'autistic': 0, 'non_autistic': 1}

    def build_transform(self, is_train):
        resize_im = self.hparams.input_size > 32
        t = []
        if is_train:
            transform = transforms.Compose([
                transforms.Resize(self.hparams.input_size, interpolation=InterpolationMode.BICUBIC),
                transforms.RandomCrop(self.hparams.input_size, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ColorJitter(brightness=self.hparams.color_jitter,
                                       contrast=self.hparams.color_jitter,
                                       saturation=self.hparams.color_jitter),
                transforms.ToTensor(),
                transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)
            ])
            if not resize_im:
                transform.transforms = transform.transforms[:-5]  # Remove Resize for small input sizes
            return transform
        else:
            t.append(transforms.Resize(int(1.0 * self.hparams.input_size), interpolation=InterpolationMode.BICUBIC))
            t.append(transforms.CenterCrop(self.hparams.input_size))
        t.append(transforms.ToTensor())
        t.append(transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD))
        return transforms.Compose(t)

    def train_dataloader(self):
        train_dataset = ImageDataset(str(self.data_path / "train"), transform=self.train_transforms,
                                     class_map=self.class_map)
        train_sampler = RandomSampler(train_dataset)
        return DataLoader(train_dataset,
                          sampler=train_sampler,
                          batch_size=self.hparams.batch_size,
                          num_workers=self.hparams.num_workers,
                          pin_memory=True,
                          persistent_workers=True,
                          drop_last=True)

    def val_dataloader(self):
        val_dataset = ImageDataset(str(self.data_path / "valid"), transform=self.eval_transforms,
                                   class_map=self.class_map)
        val_sampler = SequentialSampler(val_dataset)
        return DataLoader(val_dataset,
                          sampler=val_sampler,
                          batch_size=4,
                          num_workers=self.hparams.num_workers,
                          pin_memory=True,
                          persistent_workers=True,
                          drop_last=False)

    def test_dataloader(self):
        test_dataset = ImageDataset(str(self.data_path / "test"), transform=self.eval_transforms,
                                    class_map=self.class_map)
        test_sampler = SequentialSampler(test_dataset)
        return DataLoader(test_dataset,
                          sampler=test_sampler,
                          batch_size=4,
                          num_workers=self.hparams.num_workers,
                          pin_memory=True,
                          persistent_workers=True,
                          drop_last=False)

    def predict_dataloader(self):
        predict_dataset = ImageDataset(str(self.data_path), transform=self.eval_transforms,
                                       class_map=self.class_map)
        predict_sampler = SequentialSampler(predict_dataset)
        return DataLoader(predict_dataset,
                          sampler=predict_sampler,
                          batch_size=1,
                          num_workers=self.hparams.num_workers,
                          pin_memory=True,
                          persistent_workers=True,
                          drop_last=False)

In [ ]:
class ViTASD(nn.Module):
    def __init__(self, backbone: str, num_classes, drop_rate, drop_path_rate, input_size):
        super(ViTASD, self).__init__()
        self.num_classes = num_classes
        self.input_size = input_size

        self.backbone = create_model(
            backbone,
            pretrained=True,
            num_classes=num_classes,
            drop_rate=drop_rate,
            drop_path_rate=drop_path_rate,
            drop_block_rate=None,
            img_size=input_size
        )
        self.embed_dim = self.backbone.embed_dim

    def forward(self, x):
        x = self.backbone.patch_embed(x)
        x = x + self.backbone.pos_embed
        x = torch.cat([self.backbone.cls_token.expand(x.shape[0], -1, -1), x], dim=1)
        x = self.backbone.blocks(x)
        x = self.backbone.norm(x)
        x = x[:, 0]  # Extract the representation of the CLS token
        x = self.backbone.head(x)
        return x

In [ ]:
class ViTASDLM(LightningModule):
    def __init__(self,
                 batch_size: int = 256,
                 num_classes: int = 2,
                 epochs: int = 300,
                 attn_only: bool = False,
                 smoothing: float = 0.0,  # Label smoothing
                 vis_path: str = "./runs/vis",

                 # Model parameters
                 model: str = "deit3_base_patch16_224",  # Name of model to train
                 input_size: int = 224,  # images input size
                 drop: float = 0.0,  # Dropout rate
                 drop_path: float = 0.05,  # Drop path rate
                 pretrain_path: str = ""

                 # Optimizer parameters
                 opt: str = "adamw",
                 weight_decay: float = 0.05,

                 # Learning rate schedule parameters
                 sched: str = "cosine",
                 lr: float = 1e-4,
                 warmup_lr: float = 1e-6,
                 min_lr: float = 1e-6,
                 warmup_epochs: int = 5,  # epochs to warmup LR, if scheduler supports
                 cooldown_epochs: int = 0,  # epochs to cooldown LR at min_lr, after cyclic schedule ends

                 # Mixup parameters
                 mixup: float = 0.8,  # mixup alpha, mixup enabled if > 0
                 cutmix: float = 1.0,  # cutmix alpha, cutmix enabled if > 0.
                 mixup_prob: float = 1.0,  # Prob of performing mixup or cutmix when either/both is enabled
                 mixup_switch_prob: float = 0.5,  # Prob of switching to cutmix when both mixup and cutmix enabled
                 mixup_mode: str = "batch",  # How to apply mixup/cutmix params. Per "batch", "pair", or "elem"
                 ):

        super(ViTASDLM, self).__init__()
        self.save_hyperparameters()

        self.model: torch.nn.Module = ViTASD(
            self.hparams.model,
            num_classes=self.hparams.num_classes,
            drop_rate=self.hparams.drop,
            drop_path_rate=self.hparams.drop_path,
            input_size=self.hparams.input_size
        )

        if os.path.exists(pretrain_path):
            self._load_pretrained(pretrain_path)

        self._init_mixup()
        self._init_frozen_params()
        self.train_criterion = torch.nn.CrossEntropyLoss()
        self.valid_criterion = torch.nn.CrossEntropyLoss()
        self.valid_acc = Accuracy()
        self.auroc = AUROC(num_classes=2)
        self.confusion_matrix = ConfusionMatrix(num_classes=self.hparams.num_classes, normalize='true')

    def _init_mixup(self):
        self.mixup_fn = None
        mixup_active = self.hparams.mixup > 0 or self.hparams.cutmix > 0.
        if mixup_active:
            self.mixup_fn = Mixup(
                mixup_alpha=self.hparams.mixup,
                cutmix_alpha=self.hparams.cutmix,
                cutmix_minmax=None,
                prob=self.hparams.mixup_prob,
                switch_prob=self.hparams.mixup_switch_prob,
                mode=self.hparams.mixup_mode,
                label_smoothing=self.hparams.smoothing,
                num_classes=self.hparams.num_classes
            )

    def _load_pretrained(self, pretrain_path):
        checkpoint = torch.load(pretrain_path)
        print("Load pre-trained checkpoint from: %s" % pretrain_path)
        checkpoint_model = checkpoint['state_dict']
        state_dict = self.model.state_dict()
        for k in ['backbone.head.weight', 'backbone.head.bias']:
            if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
                print(f"Removing key {k} from pretrained checkpoint")
                del checkpoint_model[k]
        # interpolate position embedding
        interpolate_pos_embed(self.model, checkpoint_model)
        self.model.load_state_dict(checkpoint_model, strict=False)


    def _init_frozen_params(self):
        if self.hparams.attn_only:
            for name_p, p in self.model.named_parameters():
                if '.attn.' in name_p:
                    p.requires_grad = True
                else:
                    p.requires_grad = False

            self.model.backbone.head.weight.requires_grad = True
            self.model.backbone.head.bias.requires_grad = True
            self.model.backbone.pos_embed.requires_grad = True
            for p in self.model.backbone.patch_embed.parameters():
                p.requires_grad = True

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx) -> STEP_OUTPUT:
        samples, targets = batch
        if self.mixup_fn is not None:
            samples, targets = self.mixup_fn(samples, targets)
        outputs = self.forward(samples)
        loss = self.train_criterion(outputs, targets)
        loss_value = loss.item()
        self.log('Loss/train', loss_value, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx) -> STEP_OUTPUT:
        samples, targets = batch
        outputs = self.forward(samples)
        loss = self.valid_criterion(outputs, targets)
        loss_value = loss.item()
        self.valid_acc.update(outputs, targets)
        self.log("Accuracy/val", self.valid_acc, on_step=True, on_epoch=True, sync_dist=True)
        # self.log("AUROC/val", self.auroc(outputs, targets), on_epoch=True, sync_dist=True)
        self.log("Loss/val", loss_value, sync_dist=True)
        return self.valid_acc


    def test_step(self, batch, batch_idx) -> Optional[STEP_OUTPUT]:
        samples, targets = batch
        outputs = self.forward(samples)
        self.confusion_matrix.update(outputs, targets)

    def training_epoch_end(self, outputs: EPOCH_OUTPUT) -> None:
        opt: Optimizer = self.optimizers()
        self.log("LR", opt.param_groups[0]["lr"], on_epoch=True, sync_dist=True)

    def on_test_end(self) -> None:
        self.visualize_confusion_matrix()

    def configure_optimizers(self):
        optimizer = create_optimizer_v2(
            self.model,
            opt=self.hparams.opt,
            lr=self.hparams.lr,
            weight_decay=self.hparams.weight_decay,
        )
        scheduler, _ = create_scheduler(self.hparams, optimizer)
        return [optimizer], [{"scheduler": scheduler, "interval": "epoch"}]

    def lr_scheduler_step(self, scheduler: Scheduler, optimizer_idx, metric) -> None:
        scheduler.step(epoch=self.current_epoch)  # timm's scheduler need the epoch value

    def visualize_confusion_matrix(self):
        cf_matrix = self.confusion_matrix.compute().cpu()
        categories = [f'C{i}' for i in range(self.hparams.num_classes)]
        fig, ax = plt.subplots(1)
        sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='.2f', xticklabels=categories, yticklabels=categories)
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True Label')
        vis_path = Path(self.hparams.vis_path)
        fig.savefig(str(vis_path / f"cf_matrix.png"), dpi=200)


In [ ]:
# Callbacks configuration
callbacks = [
    {
        "class_path": "pytorch_lightning.callbacks.ModelCheckpoint",
        "init_args": {
            "dirpath": "/kaggle/working/",
            "save_top_k": 3,
            "monitor": "Accuracy/val",
            "mode": "max",
            "save_weights_only": True,
            "auto_insert_metric_name": True
        }
    }
]

# Instantiating the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(**callbacks[0]["init_args"])

# Logger configuration
logger_config = {
    "class_path": "pytorch_lightning.loggers.TensorBoardLogger",
    "init_args": {
        "save_dir": "/kaggle/working/lightning_logs",
        "name": "ViTASD-L",
        "version": 3,
        "default_hp_metric": False
    }
}

# Instantiating the TensorBoardLogger
logger_callback = TensorBoardLogger(**logger_config["init_args"])


In [ ]:
trainer = Trainer(
    accelerator="ddp",  # Use Distributed Data Parallel (DDP) for multi-GPU training
    gpus=[0, 1],  # Utilize GPUs with IDs 0 and 1
    min_epochs=1,
    max_epochs=300,
    precision=32,
    num_nodes=1,
    check_val_every_n_epoch=1,
    overfit_batches=0.0,
    default_root_dir=os.path.join("/kaggle", "input", "autism-gurv", "data"),
    enable_checkpointing=True,
    log_every_n_steps=30,
    enable_progress_bar=True,
    sync_batchnorm=False,
    track_grad_norm=-1,
    enable_model_summary=True,
    num_sanity_val_steps=2,
    reload_dataloaders_every_n_epochs=0,
    replace_sampler_ddp=True,
    auto_lr_find=False,
    detect_anomaly=False,
    auto_scale_batch_size=False,
    move_metrics_to_cpu=False,
    amp_backend="native",
    multiple_trainloader_mode="max_size_cycle",
    callbacks=[checkpoint_callback],  # Use the previously defined ModelCheckpoint callback
    logger=logger_callback  # Use the previously defined TensorBoardLogger
)

In [ ]:
model = ViTASDLM()

In [ ]:
dm = AutismDatasetModule()

In [ ]:
trainer.fit(model, dm)